<a href="https://colab.research.google.com/github/yeb2Binfang/ECE-GY-9163-ML-cyber/blob/main/Lab/MLSec_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE-GY 9163 ML Cyber Lab1-Email spam filtering

## Overview

In this lab, you will design an e-mail spam filter using a Naive Bayes and SVM based classification on the ling-spam dataset. You will explore the impact of feature selection and compare the performance of different variants of an NB classifier and also implement your own SVM based classifier. (Note: You may use the scikitt learn classifiers to only compare the accuracy
of their model to yours).

## Dataset

The ling-spam corpus contains e-mails from the Linguist mailing list categorized as either legitimate or spam emails. The corpus is divided into four sub-folders that contain the same emails that are pre-processed with/without lemmatization and with/without stop-word removal. The e-mails in each sub-folder partitioned into 10 "folds."

In this lab, we will use the first 9 folds from the ling-spam corpus as training data, and the $10^{th}$ fold as test data.

## What you have to do

You will implement your e-mail spam filters in Python. You are free to use any Python libraries that are relevant to the problem.

* Download the ling-spam dataset from http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz

Please use the "lingspam_public01' corpus with both lemmatization and stop-word enabled (under the lemm_stop folder).

* Your first goal is to perform feature selection using the information gain (IG) metric. From the training data, select the top-N features for N = {10, 100, 1000}. Note that feature selection based on the IG metric only accounts for the occurrence of (and not frequency with which terms appear) in the dataset.

* Next, implement the following classifiers:
  * Bernoulli NB classifier with binary features;
  * Multinomial NB with binary features, and
  * Multinomial NB with term frequency (TF) features.

* For each of the three classifiers above and for N = {10, 100, 1000} report the spam precision and spam recall. Spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam, and spam recall is defined as fraction of true spam e-mails predicted as spam. Also report the latency of each model.
* Design a Support Vector Machine (SVM) based spam filter. This problem is open ended: for instance, you can choose to use either BF or TF and any feature selection method. Note that you should NOT use the test dataset in picking the hyper-parameters of your spam filter, instead use cross-validation on the training dataset.

Finally, we covered "Adversarial Classification" (https://dl.acm.org/citation.cfm?id=1014066), an approach to update NB based e-mail spam filters in response to attacks that try to evade a basic NB filter. You are expected to implement the techniques presented in this paper. More specifically, you can make the following assumptions:
* The baseline NB classifier uses the **top-10** terms identified using the IG metric and using Boolean features.
* The adversary uses the **ADD-WORDS** strategy. Adding a term to an email incurs unit cost. The attacker seeks to find the minimum cost solution such that each spam email in the test set gets classified as legitimate by the baseline NB classifier.
* Update the baseline NB classifier in response to the attacker's strategy above. You can assume that the defender
pays a unit price for both false positives and false negatives.

## What to submit

* Your Python code in the form of a Google Colab notebook. Please also include a PDF of your colab notebook.
* Your Colab notebook should print:
* a list of the top-10 words identified from Part (1) above, and
* a list of spam precision and spam recall values for each of the three classifiers for N = {10, 100, 1000}. That is, your list should have 9 rows, one for each classifier and N combination.
* For the SVM based spam filter, please describe your methodology, i.e., what kind of features you used, how many features you used and how you selected them, the parameters of the SVM and finally the precision and recall on
the test dataset.
* For Your Neural network model, please report on the model design, input parameters, number of layers, number
of neurons in each layer and finally the accuracy and latency of the model.
* For the adversarial attack, report the False Negative rate of the baseline NB classifier before and after the attacker's modifications to test emails. Also, report the average "cost of the attacker's modifications, averaged
over all spam emails in the test set. Finally, report the False Negative and False Positive rate of the updated NBclassifier.

## Import packages

In [16]:
import numpy as np
import tarfile
import os
import string

## Load dataset

Here, we will use files that under lemm_stop folder

In [2]:
!wget "http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz" -O lingspam_public.tar.gz

--2021-10-23 17:52:44--  http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz
Resolving www.aueb.gr (www.aueb.gr)... 195.251.255.156
Connecting to www.aueb.gr (www.aueb.gr)|195.251.255.156|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www2.aueb.gr/users/ion/data/lingspam_public.tar.gz [following]
--2021-10-23 17:52:44--  http://www2.aueb.gr/users/ion/data/lingspam_public.tar.gz
Resolving www2.aueb.gr (www2.aueb.gr)... 195.251.255.138
Connecting to www2.aueb.gr (www2.aueb.gr)|195.251.255.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11564714 (11M) [application/x-gzip]
Saving to: ‘lingspam_public.tar.gz’

lingspam_public.tar 100%[===================>]  11.03M   786KB/s    in 13s     

2021-10-23 17:52:58 (853 KB/s) - ‘lingspam_public.tar.gz’ saved [11564714/11564714]



In [ ]:
!tar -xvzf "lingspam_public.tar.gz"

Lemmatisation: Lemmatisation (or lemmatization) in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form. n many languages, words appear in several inflected forms. For example, in English, the verb 'to walk' may appear as 'walk', 'walked', 'walks' or 'walking'. The base form, 'walk', that one might look up in a dictionary, is called the lemma for the word. The association of the base form with a part of speech is often called a lexeme of the word.

We will use the lemm_stop dataset. There are 10 folders totally. We need to count how many spam emails and how many ham email. We will use it later to calculate the probability

In [10]:
lemm_stop = ['lingspam_public/lemm_stop/part1/', 'lingspam_public/lemm_stop/part2/', 'lingspam_public/lemm_stop/part3/',
             'lingspam_public/lemm_stop/part4/', 'lingspam_public/lemm_stop/part5/', 'lingspam_public/lemm_stop/part6/',
             'lingspam_public/lemm_stop/part7/', 'lingspam_public/lemm_stop/part8/', 'lingspam_public/lemm_stop/part9/',
             'lingspam_public/lemm_stop/part10/']

processData function will process all txt data. The function will return a nested dict finally. It will remove all "\n","\t" and all punctuation such as '!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'. Finally, the txt data will become a list. For example: ['hello','world']. For each email, it will be put to a nest fict. Here, we have 10 folders totally, the final result for this function is 

{
  
"lingspam_public/lemm_stop/part1":

{
  
"3-565msg2.txt":[],

"3-1msg1.txt":[],...

}

}



In [34]:
def processData(dir_list):
  all_data = {}
  for dir in dir_list:
     all_data[dir]={} # a nested dict
     file_list = os.listdir(dir) # list all files in a dir   
     for file_name in file_list: # go through all files   
        f = open(dir+file_name, "r")
        f = open("lingspam_public/lemm_stop/part1/3-565msg2.txt", "r")
        data = f.read()
        data = data.lower() # make all letters to lowercase
        data = data.replace('\n','') # remove all next_line
        data = data.replace('\t','') # remove all tag
        punctuation_string = string.punctuation # remove all punctuation such as !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
        for i in punctuation_string:
          data = data.replace(i,'')
        data = data.split(' ')
        data = ' '.join(data).split()
        all_data[dir][file_name] = data # add to dict
  return all_data

In [35]:
all_data = processData(lemm_stop)

## Information gain calculation
